# Project Zeus
## Created by Michael Wentz

### Import Modules

In [ ]:
import numpy as np
import pandas as pd

### Load production data from excel file

In [ ]:
file = 'https://raw.githubusercontent.com/mwentzWW/petrolpy/master/Zeus/Sample%20Production/Sample_Prod_Data.xlsx'
well_data = pd.read_excel(file)
well_data.columns = [c.lower() for c in well_data.columns]
well_data.info()

### Check Data Layout

In [ ]:
well_data

### Delete API because there is only one well, and we don't need water

In [ ]:
well_data.drop(columns=['api number', 'water_bbl'], inplace=True)
well_data.head()

### Replace NaN with zeros for computations

In [ ]:
well_data.fillna(value=0, inplace=True)

### Calculate effective decline rate for oil and gas

In [ ]:
well_data['eff_decline_oil'] = well_data.oil_bbl.pct_change()
well_data['']
well_data['eff_decline_gas'] = well_data.gas_mcf.pct_change()

In [ ]:
well_data

### Normalize data

In [ ]:
num_days_online = len(well_data['oil_bbl'])
num_days_online

In [ ]:
days = []
for day in range(0, num_days_online):
    days.append(day)
days[0:10]

In [ ]:
well_data['time'] = days
well_data

### Calculate the type curve for oil and gas, assume b = 0.5 (will adjust later), use first 30 day average decline for di

In [ ]:
well_data.fillna(value=0, inplace=True)
well_data.loc[well_data.eff_decline_oil > 10000] = 0
well_data.loc[well_data.eff_decline_gas > 10000] = 0

In [ ]:
well_data

In [ ]:
b_decline_const = 0.1
di_oil = well_data.eff_decline_oil[0:30].mean()
di_gas = well_data.eff_decline_gas[0:30].mean()
qi_oil = well_data.oil_bbl[0:30].max()
qi_gas = well_data.gas_mcf[0:30].max()

In [ ]:
type_curve_constants = [b_decline_const, di_oil, di_gas, qi_oil, qi_gas]
type_curve_constants

In [ ]:
well_data['type_curve_oil'] = qi_oil/((1 + b_decline_const*di_oil*well_data.time)**(1/b_decline_const))
well_data['type_curve_gas'] = qi_gas/((1 + b_decline_const*di_oil*well_data.time)**(1/b_decline_const))

In [ ]:
well_data

### Plot production with type curves

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fig, ax = plt.subplots()

ax.plot(well_data.time, well_data.oil_bbl, label="Oil Production")
ax.plot(well_data.time, well_data.type_curve_oil, label="Oil Type Curve")
ax.legend(loc=2); # upper left corner
ax.set_xlabel('Time (Days)')
ax.set_ylabel('Oil (BBL)')
ax.grid(True)